## Some ideas

1. Since we already have data that labelled as same question paris. Is it possible for us to train another model which can sequetially generate a same question string depends on the input? 
2. curriculum learning
3. Ensombling:

        a. Naive RNN
        b. xgboost
        c. randomforest
        
4. Feature engineering

        a. question length (May not be very powerful)
        b. similarity metric score (Directly count it and feed it as training feature. For exampole: cosine similarity LOL)
        c. I don't think TF-IDF would work, but still worth trying if we have time.

In [2]:
import numpy as np
import pandas as pd
import re
import pickle
import json
import matplotlib.pyplot as plt
import random

from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [11]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))

duplicated_qid_pair = pickle.load(open('../dataset/processed/positive_question_id_pairs.pkl', 'rb'))
qid_question_dict = pickle.load(open('../dataset/processed/qid_question_wrods_list_dict.pkl', 'rb'))

In [5]:
def enc_question(question, enc_map):
    
    if type(question)!=str or question=="":
        return [enc_map['<ST>'], enc_map['<ED>']]
    
    def map_wrod(word):
        return enc_map[word] if word in enc_map else enc_map['<RARE>']
         
    
    # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
    separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>]|$)'
    single_word = '[\S]+' # non-empty is enough here
    
    words_list = re.findall(single_word+separator, question)
    
    return [enc_map['<ST>']] + [map_wrod(word) for word in words_list] + [enc_map['<ED>']] 
    
    
def dec_question(question, dec_map):
    return [dec_map[enc_value] for enc_value in question]

## Clip training questions

Normalize the length of all question to 30

In [12]:
for key in qid_question_dict:
    q_len = len(qid_question_dict[key])
    if q_len>=30:
        qid_question_dict[key] = qid_question_dict[key][:30]
    else:
        qid_question_dict[key] = qid_question_dict[key] + ['<ED>'] * (30-q_len)

In [14]:
pickle.dump(qid_question_dict, open('../dataset/processed/qid_question_wrods_list_dict_clipped.pkl','wb'))

## Process the validation set

In [88]:
def process_validation_samples(df, enc_map):
    
    def parse_word_list(question):
        
        if type(question)!=str:
            return []

        # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
        separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]|$)'
        single_word = '[^\s\-]+' # non-empty is enough here

        return re.findall(single_word+separator, question)
    
    # encode single question pair
    def enc_question_pair(pair):

        # This array is a local cache that identify those non-encoded rare words. 
        # We'll assign identical index (in this question pair) for each of rare words
        rare_words = []

        def enc_question(question):

            def map_word(word):
                if word in enc_map:
                    return enc_map[word]
                elif word in rare_words:
                    idx = np.where(np.array(rare_words)==word)[0][0]
                    return enc_map['<RARE' + str(idx) + '>']
                else:
                    rare_words.append(word)
                    return enc_map['<RARE' + str(len(rare_words)-1) + '>']

            word_list = parse_word_list(question)
            word_list = [map_word(word) for word in word_list]
            
            # clip the question length to 30
            q_len = len(word_list)
            if q_len>=30:
                word_list = word_list[:30]
            else:
                word_list = word_list + [enc_map['<ED>']] * (30-q_len)
            
            return embedding_matrix[np.array(word_list)]
        
        return [enc_question(pair[0]) , enc_question(pair[1])]
    
    return X,y
    
    X = np.array([enc_question_pair([series['question1'],series['question2']]) for i,series in df.iterrows()])
    y = np.array([series['is_duplicate'] for i,series in df.iterrows()])

    return [X[:,0], X[:,1]], y

In [ ]:
df_val = pickle.load(open('../dataset/processed/validation_df.pkl','rb'))
val_data = process_validation_samples(df_val, enc_map)
pickle.dump(val_data, open('../dataset/processed/validation_data.pkl','wb'))

## Parse and encode testing data (this is very slow)

I still parses question length feature, although I haven't use it.

In [17]:
import warnings
warnings.filterwarnings('ignore')

counter = 0

def change_to_list_and_clip(data, clip_st_ed=False):
    
    global counter
    counter+=1
    if counter%10000==0:
        print('Changed',counter,'data')
    
    clip_len = 30
    data['q1_len'] = len(data['question1'])
    data['q2_len'] = len(data['question2'])

    if clip_st_ed:
        data['question1'] = data['question1'][1:-1]
        data['question2'] = data['question2'][1:-1]

    if len(data['question1'])<clip_len:
        data['question1'] = data['question1'] + [enc_map['<ED>']] * (30-data['q1_len'])
    else:
        data['question1'] = data['question1'][:30]
        
    if len(data['question2'])<clip_len:
        data['question2'] = data['question2'] + [enc_map['<ED>']] * (30-data['q2_len'])
    else:
        data['question2'] = data['question2'][:30]
    
    return data

In [2]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))
df_test = pickle.load(open('../dataset/processed/processed_testing_data.pkl','rb'))

In [19]:
%%time

counter = 0
df_transformed = df_test.apply(change_to_list_and_clip, axis=1)
pickle.dump(df_transformed, open('../dataset/processed/df_test_hubertLin_version.pkl', 'wb'))

Changed 10000 data
Changed 20000 data
Changed 30000 data
Changed 40000 data
Changed 50000 data
Changed 60000 data
Changed 70000 data
Changed 80000 data
Changed 90000 data
Changed 100000 data
Changed 110000 data
Changed 120000 data
Changed 130000 data
Changed 140000 data
Changed 150000 data
Changed 160000 data
Changed 170000 data
Changed 180000 data
Changed 190000 data
Changed 200000 data
Changed 210000 data
Changed 220000 data
Changed 230000 data
Changed 240000 data
Changed 250000 data
Changed 260000 data
Changed 270000 data
Changed 280000 data
Changed 290000 data
Changed 300000 data
Changed 310000 data
Changed 320000 data
Changed 330000 data
Changed 340000 data
Changed 350000 data
Changed 360000 data
Changed 370000 data
Changed 380000 data
Changed 390000 data
Changed 400000 data
Changed 410000 data
Changed 420000 data
Changed 430000 data
Changed 440000 data
Changed 450000 data
Changed 460000 data
Changed 470000 data
Changed 480000 data
Changed 490000 data
Changed 500000 data
Changed 5

## model time!

In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import json
import matplotlib.pyplot as plt
import random

In [2]:
dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))
embedding_matrix = pickle.load(open('../dataset/processed/embedding_matrix.pkl','rb'))

train_pairs = pickle.load(open('../dataset/processed/positive_question_id_pairs.pkl', 'rb'))
qid_question_dict = pickle.load(open('../dataset/processed/qid_question_wrods_list_dict_clipped.pkl','rb'))

val_data = pickle.load(open('../dataset/processed/validation_data.pkl','rb'))

#### Train on all data directly. Overcome the imbalance problem by directly assign weights for each class label

In [3]:
test_set_pos_label_ratio = 0.1746

In [4]:
# remove validation data from training data
def holdout(qid_pair_list, ratio=0.025):
    
    def gen_suffle_idx(arr):
        random.shuffle(arr)
        return arr
    
    holdout_size = int(len(qid_pair_list)*ratio)
    idxes = gen_suffle_idx(np.arange(len(qid_pair_list)))
    holdout_idxes = idxes[:holdout_size]
    train_idxes = idxes[holdout_size:]
    
    train = qid_pair_list[train_idxes]
    val   = qid_pair_list[holdout_idxes]
    
    return train, val


In [5]:
# The validation data is heldout in the data augmenting phase, no need to holdout here

# train_pairs, val_pairs = holdout(train_pairs, ratio=0.01)

#### Calculate validation data's class weights

In [6]:
# Calculate the class weights to simulate the class disribution of testing samples
#
# These numbers and formula is got from here: 
#     https://www.kaggle.com/lystdo/quora-question-pairs/lstm-with-word2vec-embeddings#175198

test_set_pos_label_ratio = 0.1746

X_val = val_data[0]
y_val = val_data[1]
validation_pos_ratio = sum(y_val)/len(y_val)

val_weights = {
    0: (1-test_set_pos_label_ratio) / (1-validation_pos_ratio),
    1: test_set_pos_label_ratio/validation_pos_ratio
}

validation_weights = np.repeat(val_weights[1], len(y_val))
validation_weights[y_val==0] = val_weights[0]

#### Create model

In [41]:
from tensorflow.contrib.keras.python.keras.layers import Input, GRU, Reshape, Dense, Dropout, BatchNormalization, LSTM, GaussianDropout, SpatialDropout1D
from tensorflow.contrib.keras.python.keras.layers.merge import Concatenate
from tensorflow.contrib.keras.python.keras.models import Model
from tensorflow.contrib.keras.python.keras.optimizers import RMSprop, Nadam
from tensorflow.contrib.keras.python.keras.layers.wrappers import TimeDistributed
from tensorflow.contrib.keras.python.keras.layers.noise import GaussianNoise
from tensorflow.contrib.keras.python.keras.callbacks import EarlyStopping

# from keras.layers import Input, GRU, Reshape, Dense, Dropout, BatchNormalization, LSTM, GaussianDropout, Bidirectional
# from keras.layers.merge import Concatenate
# from keras.models import Model
# from keras.optimizers import RMSprop, Nadam
# # from keras.layers.wrappers import TimeDistributed
# from keras.layers.noise import GaussianNoise
# from keras.callbacks import EarlyStopping

def naive_model():

    # hyper-parameters that should be passed as function argument
    
    EMBEDDING_SIZE = 300
    vocab_size = len(enc_map)
    vocab_dim = EMBEDDING_SIZE # in my case is 300
    text_length = 30 # how many words are we going to feed in one time
    clipnorm = 1
    
    lstm_output = 168

#     q1_len = Input(shape=(1,))
#     q2_len = Input(shape=(1,))
    
    gru = LSTM(lstm_output, dropout=0.3, recurrent_dropout=0.3)
#     tdd = TimeDistributed(Dense(lstm_output)) # TimeDistributedDense(lstm_output)
    noise = GaussianNoise(0.3)
    
    # embed our encoded question to embedded vector

    encoded_question_input1 = Input(shape=(text_length,vocab_dim))
    x = noise(encoded_question_input1)
    text_embedded1 = Reshape((1,vocab_dim*text_length))(x)
    gru1 = gru(text_embedded1)
    
    # embed our encoded question to embedded vector

    encoded_question_input2 = Input(shape=(text_length,vocab_dim))
    x = noise(encoded_question_input2)
    text_embedded2 = Reshape((1,vocab_dim*text_length))(x)
    gru2 = gru(text_embedded2)
    
#     # FC without RNN, hope this patial model can deal with question with same words
#     raw_concat = Concatenate(axis=-1)([text_embedded1, text_embedded2])
#     fc_raw_concat = Dense(lstm_output//2, activation='relu')(raw_concat)
#     fc_raw_concat = Reshape((lstm_output//2,))(fc_raw_concat)

    # fully connnected
    
    x = Concatenate(axis=-1)([gru1, gru2])
    x = GaussianDropout(0.2)(x)
    x = BatchNormalization()(x)

    x = Dense(lstm_output//2, activation='relu')(x)
#     x = Concatenate(axis=-1)([x, fc_raw_concat])
#     x = MaxoutDense(lstm_output//2)(x) # ex: MaxoutDense(output_dim=1,nb_feature=3,input_dim=1)
#     x = merge([x, q1_len, q2_len], mode='concat', concat_axis=-1)
    x = GaussianDropout(0.2)(x)
    x = BatchNormalization()(x)

    out = Dense(1, activation='sigmoid')(x)
    
    # compile the model
    
    model = Model(inputs=[encoded_question_input1, encoded_question_input2], outputs=out)
    # choose objective and optimizer
#     model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-3))
    model.compile(loss='binary_crossentropy', optimizer=Nadam(lr=2e-3))
    
    return model

In [42]:
model = naive_model()

In [9]:
# Encode the list of words to list of embedded arrays
def enc_question_pairs(qid_pairs, qid_question_dict, enc_map, swap=False):

    # encode single question pair
    def enc_question_pair(pair):

        # This array is a local cache that identify those non-encoded rare words. 
        # We'll assign identical index (in this question pair) for each of rare words
        rare_words = []

        def enc_question(qid):

            def map_word(word):
                if word in enc_map:
                    return enc_map[word]
                elif word in rare_words:
                    idx = np.where(np.array(rare_words)==word)[0][0]
                    return enc_map['<RARE' + str(idx) + '>']
                else:
                    rare_words.append(word)
                    return enc_map['<RARE' + str(len(rare_words)-1) + '>']

            question = qid_question_dict[qid]
            return [map_word(word) for word in question]
        
        if swap:
            return [enc_question(pair[1]) , enc_question(pair[0])]
        else:
            return [enc_question(pair[0]) , enc_question(pair[1])]
    

    return np.array([enc_question_pair(pair) for pair in qid_pairs])

In [10]:
'''
Generate training/validation data

Pseudo code:

1. Random assign a vector for <RARE> tags, this vector values only used in this batch
2. Generate negative samples by random assigning qid pairs. (The number of negative samples is decided by "pos_ratio" parameter)
3. Encode all question pairs.
        The same RARE word in a single question pair is gauranteed to be mapped to same vector.   
4. Shuffle the order of all question pairs.
5. Embed the question pairs.
'''
def gen_batch_data(pos_pairs, pos_ratio, swap=False):
    
    global embedding_matrix
    
    def random_assign_rare_vectors():
        rare_idxes = [enc_map['<RARE'+str(i)+'>'] for i in range(30)]
        for i in rare_idxes:
            embedding_matrix[i] = np.random.rand(300)
            
    
    def gen_shuffle_idxes(num):
        a = np.arange(num)
        random.shuffle(a)
        return a
    
    def gen_negative_qid_pairs(num):
        
        '''
        TODO:
            Maybe we still need to check if generated question pair belongs to duplicated question samples.
            Although I think this is not very necessay, but we still can give it a try if we have time.
        '''
        
        id_list = list(qid_question_dict.keys())
        rnd = lambda: id_list[random.randint(0,len(id_list)-1)]
        return [ [rnd(),rnd()] for i in range(num)]
    
    # gaurantee <RARE> would be different in each batch
    random_assign_rare_vectors()
    
    # generate negative samples to match the given pos:neg ratio
    
    pos_count = len(pos_pairs)
    neg_count = int( ((1-pos_ratio)/pos_ratio) * pos_count )
    
    neg_pairs = gen_negative_qid_pairs(neg_count)
    
    pos = enc_question_pairs(pos_pairs, qid_question_dict, enc_map, swap=swap)
    neg = enc_question_pairs(neg_pairs, qid_question_dict, enc_map)
    X = np.vstack([pos,neg])
    
    # create y
    
    y = np.zeros(X.shape[0])
    y[:pos.shape[0]] = 1
    
    # shuffle
    
    shuffle_idxes = gen_shuffle_idxes(y.shape[0])
    X = X[shuffle_idxes]
    y = y[shuffle_idxes]
    
    # embed X
    
    embed_q1 = embedding_matrix[X[:,0,:]]
    embed_q2 = embedding_matrix[X[:,1,:]]
    
    return [embed_q1,embed_q2], y

In [21]:
# Calculate the class weights to simulate the class disribution of testing samples
#
# These numbers and formula is got from here: 
#     https://www.kaggle.com/lystdo/quora-question-pairs/lstm-with-word2vec-embeddings#175198

test_set_pos_label_ratio = 0.1746
training_pos_ratio = 0.3

weights = {
    0: (1-test_set_pos_label_ratio) / (1-training_pos_ratio),
    1: test_set_pos_label_ratio/training_pos_ratio
}

In [43]:
import random

batch_size = 256

def batch_generator(pairs,switch_position=False):
    
    pos_count = int(batch_size*training_pos_ratio)
    
    while True:
        
        rnd_idxes = [random.randint(0,pairs.shape[0]-1) for i in range(pos_count)]
        X, y = gen_batch_data(pairs[rnd_idxes], training_pos_ratio, swap=switch_position)
        
        yield X, y

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)
]
    
try:
#     model.fit(X_train, 
#               y_train,
#               class_weight=weights,
#               epochs=1000, 
#               batch_size=512,
#               shuffle=True,
#               validation_data=(X_val,y_val,validation_weights), 
#               callbacks=callbacks,
#               verbose=1)
    
    model.fit_generator(batch_generator(train_pairs, switch_position=False),
                        steps_per_epoch=30,
                        class_weight=weights,
                        epochs=1000,
                        validation_data=(X_val,y_val,validation_weights), 
                        callbacks=callbacks)
    
except KeyboardInterrupt:
    print('\nEarly stop by user')

Epoch 1/1000
30/30 [==============================] - 15s - loss: 0.6760 - val_loss: 0.5117
Epoch 2/1000
30/30 [==============================] - 12s - loss: 0.4754 - val_loss: 0.4715
Epoch 3/1000
30/30 [==============================] - 12s - loss: 0.4171 - val_loss: 0.4765
Epoch 4/1000
30/30 [==============================] - 12s - loss: 0.3899 - val_loss: 0.4778
Epoch 5/1000
 4/30 [===>..........................] - ETA: 6s - loss: 0.3498
Early stop by user


Calculate the ROC_AUC score

In [39]:
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

pred = model.predict(X_val)

# pred_bin = np.zeros(len(pred))
# pred_bin[(pred>0.5).reshape(len(pred))] = 0.999999
# pred_bin[(pred<=0.5).reshape(len(pred))] = 0.000001

pred[pred>0.999] = 0.999999
pred[pred<=0.001] = 0.000001

print('log_loss=',log_loss(y_val, pred, sample_weight=validation_weights))

print('roc_auc=',roc_auc_score(y_val, pred, sample_weight=validation_weights))

log_loss= 0.534604601134
roc_auc= 0.62890797396


In [33]:
model_name = 'HubertLin_naive_LSTM_dynamic_training'
model.save('../model/'+model_name+'.model')

## customized testcase

In [50]:
# from keras.models import load_model
# model = load_model('../model/HubertLin_naive_LSTM_alot_dropouts.model')

In [25]:
def enc_question_pair(question_pair_list, enc_map, capsulate_st_ed=False):

    def enc_question(question):

        if type(question)!=str or question=="":
            if capsulate_st_ed:
                return [enc_map['<ST>'], enc_map['<ED>']]
            else:
                return []

        def map_wrod(word):
            if word in enc_map:
                return enc_map[word]
            elif word in rare_words:
                idx = np.where(np.array(rare_words)==word)[0][0]
                return enc_map['<RARE' + str(idx) + '>']
            else:
                rare_words.append(word)
                return enc_map['<RARE' + str(len(rare_words)-1) + '>']

        # identify special characters that separate words : (space) ' ! " ? @ ^ + * / . , ~ ( ) [ ] { } & | ` $ % = : ; < >  
        separator = '(?=[\s\'!"?@\^+*/\.,~\(\)\[\]\{\}\&\|`\$\%\=:;\<\>\-]|$)'
        single_word = '[^\s\-]+' # non-empty is enough here

        words_list = re.findall(single_word+separator, question)

        if capsulate_st_ed:
            return [enc_map['<ST>']] + [map_wrod(word) for word in words_list] + [enc_map['<ED>']] 
        else:
            return [map_wrod(word) for word in words_list]
        
    # This array is a local cache that identify those non-encoded rare words. 
    # We'll assign identical index (in this question pair) for each of rare words
    rare_words = []
    
    return enc_question(question_pair_list[0]) , enc_question(question_pair_list[1])
    
    
def dec_question(question, dec_map):
    return [dec_map[enc_value] for enc_value in question]

def clip_length(data, clip_st_ed=False):
    
    clip_len = 30
    
    if clip_st_ed:
        data[0] = eval(data[0])[1:-1]
        data[1] = eval(data[1])[1:-1]
    
    if len(data[0])<clip_len:
        data[0] = data[0] + [enc_map['<ED>']] * (30-len(data[0]))
    else:
        data[0] = data[0][:30]
        
    if len(data[1])<clip_len:
        data[1] = data[1] + [enc_map['<ED>']] * (30-len(data[1]))
    else:
        data[1] = data[1][:30]
        
    return data

In [40]:
import numpy as np
import pandas as pd
import re
import pickle

dec_map = pickle.load(open('../dataset/processed/dec_map.pkl','rb'))
enc_map = pickle.load(open('../dataset/processed/enc_map.pkl','rb'))

testcases = [
    {
        'q1': 'I am a good man',
        'q2': 'A good man is me'
    },
    {
        'q1': 'The <RARE0> is totally fucking awful',
        'q2': 'The <RARE0> is the best thing I have ever seen'
    },
    {
        'q1': 'The <RARE0> is the best thing I have ever seen',
        'q2': 'The <RARE0> is totally fucking awful',
    },
    {
        'q1': 'How <RARE0> can be used in cooking',
        'q2': 'Can we find <RARE1> on the moon'
    },
    {
        'q1': 'A bird is flying in the sky',
        'q2': 'My bird is flying over the ocean'
    },
    {
        'q1': 'What is your favorite food',
        'q2': 'Which is your favorite one'
    },
    {
        'q1': '<RARE0> <RARE1> <RARE2> <RARE3> <RARE4> <RARE5> ',
        'q2': '<RARE6> <RARE7> <RARE8> <RARE9> <RARE10> <RARE11> ',
    },
    {
        'q1': '<RARE0> <RARE1> <RARE2> <RARE3> <RARE4> <RARE5> ',
        'q2': '<RARE0> <RARE1> <RARE2> <RARE3> <RARE4> <RARE5> ',
    },
    {
        'q1': ' what a <RARE0> program I have never seen such beatiful code ever',
        'q2': ' what a <RARE0> program I have never seen such beatiful code ever',
    },
    {
        'q1': ' what a <RARE0> program I have never seen such beatiful code ever',
        'q2': ' what a <RARE1> program I have never seen such beatiful code ever',
    },
]

for testcase in testcases:
    
    enc1, enc2 = enc_question_pair([testcase['q1'],testcase['q2']], enc_map)
    enc1, enc2 = clip_length([enc1,enc2])
    print(' '.join(dec_question(enc1,dec_map)))
    print(' '.join(dec_question(enc2,dec_map)))

    emb1 = embedding_matrix[np.array([enc1])]
    emb2 = embedding_matrix[np.array([enc2])]

    pred = model.predict([emb1,emb2])
    print('Predicting is same question proba =', pred)
    print('')


I am a good man <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
A good man is me <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Predicting is same question proba = [[ 0.54771608]]

The <RARE0> is totally fucking awful <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
The <RARE0> is the best thing I have ever seen <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Predicting is same question proba = [[ 0.1880715]]

The <RARE0> is the best thing I have ever seen <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
The <RARE0> is totally fucking awful <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Pred

## Try to predict some testing data

In [40]:
df_test = pickle.load(open('../dataset/processed/df_test_hubertLin_version.pkl', 'rb'))

In [46]:
a = df_test.ix[186461]
q1 = dec_question(a['question1'], dec_map)
q2 = dec_question(a['question2'], dec_map)
print(q1)
print(q2)

emb1 = embedding_matrix[np.array([a['question1']])]
emb2 = embedding_matrix[np.array([a['question2']])]
pred = model.predict([emb1, emb2])
print('\nPredicting is same question proba =', pred)

['What', 'is', 'customer', 'lifetime', '<RARE0>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>']
['What', 'is', 'customer', 'lifetime', 'value', 'in', '<RARE1>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>', '<ED>']

Predicting is same question proba = [[ 0.60896122]]


## Prediction time

In [ ]:
from keras.models import load_model

try:
    model==None
except:
    model = load_model('../model/'+model_name+'.model')

df_test = pickle.load(open('../dataset/processed/df_test_hubertLin_version.pkl', 'rb'))

In [47]:
%%time

partition_size = 10000

def predict(i,q):
    
    if i%10 == 0:
        print(i*partition_size, '/', len(df_test))
    
    x1 = np.array(list(q['question1']))
    x2 = np.array(list(q['question2']))
    
    x1 = embedding_matrix[x1]
    x2 = embedding_matrix[x2]
    
    return model.predict([x1,x2])

partition_len = len(df_test)//partition_size +1
result = [predict(i,df_test.iloc[i*partition_size:(i+1)*partition_size]) for i in range(partition_len)]
con = np.concatenate(result)
df_result = pd.DataFrame({'test_id':np.arange(len(con)),'is_duplicate':con.reshape(len(con))}, columns=['test_id','is_duplicate'])
df_result.to_csv('../result/prediction.csv', index=False)

0 / 2345796
100000 / 2345796
200000 / 2345796
300000 / 2345796
400000 / 2345796
500000 / 2345796
600000 / 2345796
700000 / 2345796
800000 / 2345796
900000 / 2345796
1000000 / 2345796
1100000 / 2345796
1200000 / 2345796
1300000 / 2345796
1400000 / 2345796
1500000 / 2345796
1600000 / 2345796
1700000 / 2345796
1800000 / 2345796
1900000 / 2345796
2000000 / 2345796
2100000 / 2345796
2200000 / 2345796
2300000 / 2345796
Wall time: 48min 13s


In [51]:
if len(df_result)!=2345796:
    print('Your result prediction count is not fit to the testing data length 2345796 , yours:', len(df_result))
else:
    print('Prediction success')

Prediction success


## visualize the prediction result

In [54]:
over = df_test[df_result['is_duplicate']>0.8]

c = 0
for i,s in over.iterrows():
    print(' '.join(dec_question(s['question1'],dec_map)))
    print(' '.join(dec_question(s['question2'],dec_map)))
    print(df_result.ix[i]['is_duplicate'])
    print('')
    c+=1
    if c>5:
        break

Can a vacuum cleaner concentrate suck your eye out if it is pressed against your <RARE0> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Could a vacuum cleaner suck get your eye out if directly pressed on the <RARE0> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
0.813005

Is web development just building <RARE0> best you get a web developer job if you know how to make a <RARE1> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
Is web development just building <RARE0> Can you get a backend web developer job if you know how year make a <RARE1> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
0.966372

How do I overcome my shyness with <RARE0> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
How do you overcome being <RARE1> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED> <ED>
0.865856

If I like a comment to a computer post b